reference: https://www.quantopian.com/tutorials/pipeline#lesson12

Before we go crazy, we ultimately need the following:
1. initialize:
    - Schedule our rebalance function to run at the start of each week, when the market opens. 
    - Record variables at the end of each day
    - create pipeline and attach it to our algorithm


2. make_pipeline:
    - define base universe
    - get the factors/characteristics/masks/filters that you need
    - return a pipeline that contains information/characters that you want


3. compute_target_weights:
    - initialize target weights with a dictionary
    - if there are securities in our long and short lists, then give them an even target weights
    - if there are securities not in our long and short lists, then exit position
    - assign long and short weights
    
    
4. before_trading_start:
    - get the pipeline ready
    - go long for securities that should be long, and check if they can be traded
    - go short for securities that should be short, and check if they can be traded


5. my_rebalance:
    - calculate target weights to rebalance


6. my_record_vars:
    - record what you've done. typically leverage, long & short counts


### Moving to the IDE

So far, we have created and run a pipeline in research. Now, let's move to the IDE. To start out, let's create a skeleton algorithm, import `Pipeline`, and add a `make_pipeline` function to it that creates an empty pipeline. 

In [ ]:
from quantopian.pipeline import Pipeline

def initialize(context):
    my_pipe = make_pipeline()

def make_pipeline():
    return Pipeline()

### Attaching a Pipeline

Recall that in research, we ran `make_pipeline` to create an instance of our pipeline object and we used `run_pipeline` to run our pipeline over a specified date range. We cannot safely do this in an algorithm, we have to somehow allow the simulation to run our pipeline for us. In order for the simulation to run our pipeline, we must attach our pipeline with `attach_pipeline`.

The `attach_pipeline` function requires two arguments: a reference to our `Pipeline` object, and a `string` name for the pipeline which can be decided arbitrarily. Let's import `attach_pipeline` and attach our empty pipeline in our skeleton example. 

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.algorithm import attach_pipeline

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

Now that our pipeline is attached, it will be run once each day of the simulation. If an algorithm is backtested or live traded from Monday, June 6, 2016 to Friday, Jun 10, 2016, our pipeline will be run once on each day of the week (5 times total). Attaching our pipeline will produce a new output dataframe each day. The daily output is similar to the output of `run_pipeline` in research. However, the output dataframe in our algorithm will not include the date in the index, as the current simulation date is implied to be the date of our pipeline computations. 

### Pipeline Output

The daily output of our pipeline can be retrieved using `pipeline_output` in `before_trading_start`. `pipeline_output` requires the name of the attached pipeline as an argument and returns the output dataframe for the current date in the simulation. Let's import `pipeline_output` and modify our skeleton example to store our pipeline output in `context` each day. 

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.algorithm import attach_pipeline, pipeline_output

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

def before_trading_start(context, data):
    # Store our pipeline output DataFrame in context.
    context.output = pipeline_output('my_pipeline')

Our skeleton example now produces an empy dataframe with 8000+ rows and 0 columns each day. The output dataframe will look like this (note that the index is no longer a `MultiIndex` as it was in research): 

<img style="float: left;" src="equity_list.png">

### Using Our Pipeline From Research

To include the pipeline that we created in the previous lesson in our algorithm, we can simply copy the `make_pipeline` function that we wrote in research to our algorithm as well as the required `import` statements. The following will run our pipeline and store the output dataframe with 150 rows and 2 columns (`longs` and `shorts`) in `context` each day. 

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.filters.fundamentals import Q1500US

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    """
    Create our pipeline.
    """

    # Base universe set to the QTradableStocksUS.
    base_universe = QTradableStocksUS()

    # 10-day close price average.
    mean_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=10,
        mask=base_universe
    )

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=30,
        mask=base_universe
    )

    percent_difference = (mean_10 - mean_30) / mean_30

    # Filter to select securities to short.
    shorts = percent_difference.top(75)

    # Filter to select securities to long.
    longs = percent_difference.bottom(75)

    # Filter for all securities that we want to trade.
    securities_to_trade = (shorts | longs)

    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts
        },
        screen=(securities_to_trade),
    )

def before_trading_start(context, data):
    # Store our pipeline output DataFrame in context
    context.output = pipeline_output('my_pipeline')

The output from this pipeline will look something like this every day: 

<img style="float: left;" src="equity_long_short.png">

We can then define some functions to compute target weights and place orders for our long and short positions as specified by our pipeline output. Let's use some of the basics that we learned in the Getting Started Tutorial to implement weight calculations and ordering. 

In [ ]:
def compute_target_weights(context, data):
    """
    Compute ordering weights.
    """

    # Initialize empty target weights dictionary.
    # This will map securities to their target weight.
    weights = {}
    
    # If there are securities in our longs and shorts lists,
    # compute even target weights for each security. 
    if context.longs and context.shorts:
        long_weight = 0.5 / len(context.longs)
        short_weight = -0.5 / len(context.shorts)
    else:
        return weights
    
    # Exit positions in our portfolio if they are not
    # in our longs or shorts lists. 
    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            weights[security] = 0

    for security in context.longs:
        weights[security] = long_weight

    for security in context.shorts:
        weights[security] = short_weight

    return weights

def before_trading_start(context, data):
    """
    Get pipeline results.
    """

    # Gets our pipeline output every day.
    pipe_results = pipeline_output('my_pipeline')

    # Go long in securities for which the 'longs' value is True,
    # and check if they can be traded. 
    context.longs = []
    for sec in pipe_results[pipe_results['longs']].index.tolist():
        if data.can_trade(sec):
            context.longs.append(sec)

    # Go short in securities for which the 'shorts' value is True,
    # and check if they can be traded.
    context.shorts = []
    for sec in pipe_results[pipe_results['shorts']].index.tolist():
        if data.can_trade(sec):
            context.shorts.append(sec)

def my_rebalance(context, data):
    """
    Rebalance weekly.
    """
    
    # Calculate target weights to rebalance
    target_weights = compute_target_weights(context, data)
    
    # If we have target weights, rebalance our portfolio
    if target_weights:
        order_optimal_portfolio(
            objective=opt.TargetWeights(target_weights),
            constraints=[],
        )

Finally, let's put everything together and record some variables. We'll specify our algorithm to rebalance weekly. 

In [ ]:
from quantopian.algorithm import order_optimal_portfolio
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.filters import QTradableStocksUS
import quantopian.optimize as opt

def initialize(context):
    # Schedule our rebalance function to run at the start of 
    # each week, when the market opens. 
    schedule_function(
        my_rebalance,
        date_rules.week_start(),
        time_rules.market_open()
    )

    # Record variables at the end of each day.
    schedule_function(
        my_record_vars,
        date_rules.every_day(),
        time_rules.market_close()
    )

    # Create our pipeline and attach it to our algorithm.
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    """
    Create our pipeline.
    """

    # Base universe set to the QTradableStocksUS.
    base_universe = QTradableStocksUS()

    # 10-day close price average.
    mean_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=10,
        mask=base_universe
    )

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=30,
        mask=base_universe
    )

    percent_difference = (mean_10 - mean_30) / mean_30

    # Filter to select securities to short.
    shorts = percent_difference.top(75)

    # Filter to select securities to long.
    longs = percent_difference.bottom(75)

    # Filter for all securities that we want to trade.
    securities_to_trade = (shorts | longs)

    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts
        },
        screen=(securities_to_trade),
    )

def compute_target_weights(context, data):
    """
    Compute ordering weights.
    """

    # Initialize empty target weights dictionary.
    # This will map securities to their target weight.
    weights = {}
    
    # If there are securities in our longs and shorts lists,
    # compute even target weights for each security. 
    if context.longs and context.shorts:
        long_weight = 0.5 / len(context.longs)
        short_weight = -0.5 / len(context.shorts)
    else:
        return weights
    
    # Exit positions in our portfolio if they are not
    # in our longs or shorts lists. 
    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            weights[security] = 0

    for security in context.longs:
        weights[security] = long_weight

    for security in context.shorts:
        weights[security] = short_weight

    return weights

def before_trading_start(context, data):
    """
    Get pipeline results.
    """

    # Gets our pipeline output every day.
    pipe_results = pipeline_output('my_pipeline')

    # Go long in securities for which the 'longs' value is True,
    # and check if they can be traded. 
    context.longs = []
    for sec in pipe_results[pipe_results['longs']].index.tolist():
        if data.can_trade(sec):
            context.longs.append(sec)

    # Go short in securities for which the 'shorts' value is True,
    # and check if they can be traded.
    context.shorts = []
    for sec in pipe_results[pipe_results['shorts']].index.tolist():
        if data.can_trade(sec):
            context.shorts.append(sec)

def my_rebalance(context, data):
    """
    Rebalance weekly.
    """
    
    # Calculate target weights to rebalance
    target_weights = compute_target_weights(context, data)
    
    # If we have target weights, rebalance our portfolio
    if target_weights:
        order_optimal_portfolio(
            objective=opt.TargetWeights(target_weights),
            constraints=[],
        )

def my_record_vars(context, data):
    """
    Record variables at the end of each day.
    """

    longs = shorts = 0
    for position in context.portfolio.positions.itervalues():
        if position.amount > 0:
            longs += 1
        elif position.amount < 0:
            shorts += 1

    # Record our variables.
    record(
        leverage=context.account.leverage,
        long_count=longs,
        short_count=shorts
    )

Note: When a pipeline is run in a backtest, the computations are performed in batches to increase overall computation speed. Because the computations are performed in batches, the performance chart will appear to pause periodically.

Congratulations on completing the Pipeline tutorial! Try designing a pipeline in research and use it in an algorithm of your own. 